# Project 1: Machine Learning

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from typing import Tuple
from helper import load_csv_data
from processing import *
from implementations import *
from feature_expansion import *
from crossvalidation import *
from metrics import f1_score, mse_loss

# Import of the data
Import of the train and test data

In [2]:
y_train, x_train, id_train = load_csv_data("../data/train.csv")
y_test , x_test , id_test  = load_csv_data("../data/test.csv")
# print('train data shape: ', x_train.shape, y_train.shape)
# print('test  data shape: ', x_test.shape, y_test.shape)
with open('../col_name.json', 'r') as file:
    features = json.load(file)['col_names']

# Preprocess data
We pre process the data to get a clean dataset

In [3]:
x_train_cleaned = standardize(clean_data(x_train, features))

We then divide the dataset depending on the Pri_Jet_number feature which can take values 0, 1, 2 or 3. Since the number values that are equal to 3 is really small, we will combine it with the values which have 2 so we will have a 3 subsets

### Feature expansion
We will now do feature engineering to increase the results we will have. We do degree root transformation, polynomial transformation, logarithmic transformation and reciprocical transformation.

In [ ]:
x_train_finished = build_new_x(x_train_cleaned)

# Cross-Validation Pipeline

cross validation pipeline. we will use 5-fold cross validation for choosing the optimal parameters.
below is the list of models we will use throughout this process.

1. least_squares (no parameter tuning needed.)
2. ridge LS 
3. mse_gd
4. mse_sgd
5. logistic
6. reg_logistic

we will optimise the weigth based on the mse loss. But the selection process will be based on observing F1 score on validation set.

To see the effect of data manipulation, we will try 3 sets of data.
1. cleaned data
2. standardized data
3. feature-engineered data

From this process, we would expect the 3rd trial would give us the best result.

In [38]:
from itertools import product

class HyperParameterTuner:
    def __init__(
        self, 
        x: np.ndarray, 
        y: np.ndarray, 
        model_name: str,
        num_folds: int=5,
        num_seed: int=0,
        max_iter: int=10,
        grid_size: int=10
    ):

        available_models = {
            'least_squares': least_squares,
            'ridge': ridge_regression,
            'mse_gd': mean_squared_error_gd,
            'mse_sgd': mean_squared_error_sgd,
            'logistic': logistic_regression,
            'reg_logistic': reg_logistic_regression
        }

        self.x = x
        self.y = y
        self.model = available_models[model_name]
        self.model_name = model_name
        self.num_folds = num_folds
        self.num_seed  = num_seed
        self.max_iter  = max_iter
        self.grid_size = grid_size
        
        # build k_indices
        np.random.seed(self.num_seed)
        self.build_k_indices()

        # get model params given model specs.
        model_parameters = {
            'least_squares': {}, 
            'ridge'        : {'lambda_': None},
            'mse_gd'       : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None},
            'mse_sgd'      : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None},
            'logistic'     : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None},
            'reg_logistic' : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None, 'lambda_': None},
        }
        self.hyp_params = model_parameters[model_name]

    def tune_(self) -> Tuple[list, float]:
        """
        hyperparameter tuning done by grid search.
        best parameters are found by finding the maximum f1 scores.
        """

        lambdas = np.logspace(-15,    2, self.grid_size)
        gammas  = np.logspace(-6 ,-0.25, self.grid_size)     
        
        f1_scores = np.array([])
        params = self.hyp_params
        
        if self.model_name == 'least_squares':
            results = np.array([[k, self.cross_validation_per_k(k, self.hyp_params)[-1]] for k in range(self.num_folds)])
            return results[np.argmax(results[:,-1])]   
        
        elif self.model_name == 'reg_logistic':
            lambda_and_gammas = list(product(gammas, lambdas))
            for i, (gamma, lambda_) in enumerate(lambda_and_gammas):
                params['gamma'], params['lambda_'] = gamma, lambda_
                results = np.array([self.cross_validation_per_k(k, params) for k in range(self.num_folds)])
                f1_scores = np.append(f1_scores, np.mean(results, axis=0)[-1])
                if i %10 == 0:
                    print(f'Progress {i}/{self.grid_size**2}, lambda_: {lambda_},  gamma: {gamma}, f1: {np.round(np.mean(results, axis=0)[-1], 4)}')            
                       
            f1_scores[np.isnan(f1_scores)] = 0 
            optimum_idx = np.argmax(f1_scores)
            print(optimum_idx, len(f1_scores), len(list(lambda_and_gammas)))
            best_params, best_f1 = lambda_and_gammas[optimum_idx], f1_scores[optimum_idx]
            return {'params' :best_params, 'f1_score' :best_f1}     

        elif self.model_name == 'ridge':
            for i, lambda_ in enumerate(lambdas):
                params['lambda_'] =lambda_
                results = np.array([self.cross_validation_per_k(k, params) for k in range(self.num_folds)])
                f1_scores = np.append(f1_scores, np.mean(results, axis=0)[-1])
                if i %10 == 0:
                    print(f'Progress {i}/{len(lambdas)}, lambda_: {lambda_}, f1: {np.round(np.mean(results, axis=0)[-1], 4)}')            
            
            f1_scores[np.isnan(f1_scores)] = 0 
            optimum_idx = np.argmax(f1_scores)
            best_params, best_f1 = lambdas[optimum_idx], f1_scores[optimum_idx]
            return {'lambda_' :best_params, 'f1_score' :best_f1}     
        
        else:
            for i, gamma in enumerate(gammas):
                params['gamma'] = gamma
                results = np.array([self.cross_validation_per_k(k, params) for k in range(self.num_folds)])
                f1_scores = np.append(f1_scores, np.mean(results, axis=0)[-1])
                if i %10 == 0:
                    print(f'Progress {i}/{len(gammas)}, gamma :{gamma} f1: {np.round(np.mean(results, axis=0)[-1], 4)}')
            # set nan values to 0.
            f1_scores[np.isnan(f1_scores)] = 0 
            
            # get optimum index
            optimum_idx = np.argmax(f1_scores)
            best_params, best_f1 = gammas[optimum_idx], f1_scores[optimum_idx]
            return {'gamma' :best_params, 'f1_score' :best_f1}
        
    def cross_validation_per_k(self, k: int, params: dict) -> list:
        """return the loss of given model."""
        # get k'th subgroup in test, others in train
        tr_indices, te_indices = self.k_indices[~(np.arange(self.k_indices.shape[0]) == k)].reshape(-1),\
                                 self.k_indices[k]
        
        # split the data based on train and validation indices
        y_trn, y_val = self.y[tr_indices], self.y[te_indices]
        x_trn, x_val = self.x[tr_indices], self.x[te_indices]

        # run the model
        params['tx'], params['y'] = x_trn, y_trn
        w, _ = self.model(**params)
        
        # calculate the loss for train and test data
        loss_trn = np.sqrt(mse_loss(y_trn, x_trn, w))
        loss_val = np.sqrt(mse_loss(y_val, x_val, w))
        
        # get validation f1-score
        y_pred = get_classification_pred(x_val, w)
        f1_val = f1_score(y_val, y_pred)
        return [loss_trn, loss_val, f1_val]

    def build_k_indices(self):
        """
        build k indices for k-fold.
        Args:
            y:      shape=(N,)
            k_fold: K in K-fold, i.e. the fold num
            seed:   the random seed

        Returns:
            A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold
        """
        num_row  = self.y.shape[0]
        interval = int(num_row / self.num_folds)
        indices  = np.random.permutation(num_row)

        self.k_indices = np.array([indices[k * interval : (k + 1) * interval] for k in range(self.num_folds)])
   


In [25]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'least_squares', max_iter=100, grid_size=10)
tuner.tune_()

array([0.        , 0.66636489])

In [26]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'mse_gd', max_iter=50, grid_size=20)
tuner.tune_()

Progress 0/20, gamma :1e-06 f1: 0.44
Progress 10/20, gamma :0.001062467830894041 f1: 0.4363


/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/implementations.py:135: RuntimeWarning: overflow encountered in exp
  return np.exp(t) / (1 + np.exp(t))
/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/implementations.py:135: RuntimeWarning: invalid value encountered in divide
  return np.exp(t) / (1 + np.exp(t))
/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/metrics.py:30: RuntimeWarning: invalid value encountered in long_scalars
  precission = true_positive / (true_positive + false_positive)


{'gamma': 0.280135676119887, 'f1_score': 0.6626473443870915}

In [27]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'mse_sgd', max_iter=50, grid_size=20)
tuner.tune_()

Progress 0/20, gamma :1e-06 f1: 0.44
Progress 10/20, gamma :0.001062467830894041 f1: 0.4318


/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/implementations.py:135: RuntimeWarning: overflow encountered in exp
  return np.exp(t) / (1 + np.exp(t))
/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/implementations.py:135: RuntimeWarning: invalid value encountered in divide
  return np.exp(t) / (1 + np.exp(t))
/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/metrics.py:30: RuntimeWarning: invalid value encountered in long_scalars
  precission = true_positive / (true_positive + false_positive)


{'gamma': 2.007389180054629e-06, 'f1_score': 0.43999712896575083}

In [40]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'ridge', max_iter=50, grid_size=25)
tuner.tune_()

Progress 0/25, lambda_: 1e-15, f1: 0.6648
Progress 10/25, lambda_: 1.2115276586285901e-08, f1: 0.6647
Progress 20/25, lambda_: 0.14677992676220736, f1: 0.6537


{'lambda_': 1e-15, 'f1_score': 0.6647511072771948}

In [29]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'logistic', max_iter=50, grid_size=20)
tuner.tune_()

Progress 0/20, gamma :1e-06 f1: 0.44
Progress 10/20, gamma :0.001062467830894041 f1: 0.441


{'gamma': 0.5623413251903491, 'f1_score': 0.6381401682827477}

In [39]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'reg_logistic', max_iter=10, grid_size=10)
tuner.tune_()

Progress 0/100, lambda_: 1e-15,  gamma: 1e-06, f1: 0.44
Progress 10/100, lambda_: 1e-15,  gamma: 4.354004653656653e-06, f1: 0.44
Progress 20/100, lambda_: 1e-15,  gamma: 1.8957356524063752e-05, f1: 0.44
Progress 30/100, lambda_: 1e-15,  gamma: 8.254041852680174e-05, f1: 0.44
Progress 40/100, lambda_: 1e-15,  gamma: 0.00035938136638046257, f1: 0.4401
Progress 50/100, lambda_: 1e-15,  gamma: 0.001564748141658019, f1: 0.4403
Progress 60/100, lambda_: 1e-15,  gamma: 0.006812920690579608, f1: 0.4412
Progress 70/100, lambda_: 1e-15,  gamma: 0.02966348839177725, f1: 0.4453
Progress 80/100, lambda_: 1e-15,  gamma: 0.12915496650148828, f1: 0.4638
Progress 90/100, lambda_: 1e-15,  gamma: 0.5623413251903491, f1: 0.5525
88 100 100


{'params': (0.12915496650148828, 1.2915496650148828),
 'f1_score': 0.5947337865658154}

# Learning algorithms

### Least squares

In [21]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [ ]:
weight, loss = least_squares(y_train, x_train_cleaned)
y_pred = get_classification_pred(x_train_cleaned, weight)
print(f1_score(y_train, y_pred))

### Least squares with ridges regression

In [ ]:
weight, loss = ridge_regression(y_train, x_train_cleaned, 10)
print(loss)

### Least squares with gradient descent

In [ ]:
weight, loss = mean_squared_error_gd(y_train, x_train_cleaned, np.ones((31,)), 100, 1e-3)
loss

### Least squares with stochastic gradient descent

In [ ]:
weight, loss = mean_squared_error_sgd(y_train, x_train_cleaned, np.ones((31,)), 100, 1e-3)
loss

### Logistic regression

In [ ]:
w, l = logistic_regression(y_train, x_train_cleaned, np.ones((31,)), 1000, 1e-3)

In [ ]:
w, l = reg_logistic_regression(y_train, x_train_cleaned, 0.2,  np.ones((31,)), 1000, 1e-3)

In [35]:
lambdas = np.logspace(-15,    2, 10)
gammas  = np.logspace(-6 ,-0.25, 10)     
lambda_and_gammas = product(gammas, lambdas)
a = list(lambda_and_gammas)[99]
a

(0.5623413251903491, 100.0)